# Introduction / problématique
On souhaite analyser les connexions au réseau d'une université autrichienne.
Un routeur Netflow a effectué une surveillance sur le réseau et a restitué des données Netflow contenues dans le fichier dataset.netflow.

Il semblerait qu'une attaque ait été perpetrée. Il faut détecter l'adresse IP de la machine qui a subi cette attaque.



# Import des données et des bibliothèques

In [1]:
import pandas as pd
from pandas import Grouper
import numpy as np
import csv

In [2]:
import datetime as dt
from datetime import *
import time

In [3]:
import itertools
from itertools import groupby

In [5]:
#importer les données dans un dataframe nommé df
df = pd.read_csv('dataset.netflow', index_col=False, low_memory=False)

# Exploration des données

In [ ]:
#dimension du dataframe
print('La dimension du dataframe est : ', df.shape)

In [ ]:
#afficher les 5 premières lignes du dataframe
df.head()

In [ ]:
#type de chacune des variables:
df.dtypes

# Manipulation / transformation des données

# Nettoyage du dataframe

## Renommer les variables

Cela permet de les manipuler lors de l'utilisation de focntions diverses

In [6]:
#renommer la colonne "Date flow start" pour retirer les espaces
df.rename(columns={'Date flow start':'Datetime'},inplace = True)
df.head()

,Datetime,Durat,Prot,dir,Flags,Tos,Packets,Bytes,Flows,SourceIP,SourcePort,DestIP,DestPort
0,2011-08-18 10:19:13.328,0.002,TCP,->,FRPA_,0.0,4.0,321.0,1.0,147.32.86.166,33426,212.24.150.110,25443
1,2011-08-18 10:19:13.328,4.995,UDP,->,INT,0.0,617.0,40095.0,1.0,82.39.2.249,41915,147.32.84.59,43087
2,2011-08-18 10:19:13.329,4.996,UDP,->,INT,0.0,1290.0,1909200.0,1.0,147.32.84.59,43087,82.39.2.249,41915
3,2011-08-18 10:19:13.330,0.000,TCP,->,A_,0.0,1.0,66.0,1.0,147.32.86.166,42020,147.32.192.34,993
4,2011-08-18 10:19:13.330,0.000,TCP,->,FPA_,0.0,2.0,169.0,1.0,212.24.150.110,25443,147.32.86.166,33426


On remarque qu'il y a un espace avant Packets ce qui nous posera problème au moment de la manipulation de cette colonne.

In [7]:
#retrait de l'espace avant "Packets"
df.rename(columns={'  Packets':'Packets'},inplace = True)
df.count()

Datetime      5180851
Durat         5179568
Prot          5179568
dir           5179568
Flags         5179568
Tos           5179568
Packets       5179568
Bytes         5179568
Flows         5179568
SourceIP      5179568
SourcePort    4806834
DestIP        5179568
DestPort      4804903
dtype: int64

## Retrait des variables inutiles

Etant donné que l'on souhaite trouver une anomalie au niveau des connexions, je ne garde que les sources d'information relatives à cette recherche, à savoir: "IP Source", "IP de destination", "Port de destination", "nombre de paquets émis" et bien entendu les "dates" et surtout "heures" d'émission des paquets.

In [8]:
df_drop = df.drop(['Flags','SourcePort', 'Tos', 'Flows', 'dir'], axis=1)
print("Il ne reste plus que: ", len(df_drop.columns), "variables dans le dataframe: ")
df_drop.head()

Il ne reste plus que:  8 variables dans le dataframe: 


,Datetime,Durat,Prot,Packets,Bytes,SourceIP,DestIP,DestPort
0,2011-08-18 10:19:13.328,0.002,TCP,4.0,321.0,147.32.86.166,212.24.150.110,25443
1,2011-08-18 10:19:13.328,4.995,UDP,617.0,40095.0,82.39.2.249,147.32.84.59,43087
2,2011-08-18 10:19:13.329,4.996,UDP,1290.0,1909200.0,147.32.84.59,82.39.2.249,41915
3,2011-08-18 10:19:13.330,0.000,TCP,1.0,66.0,147.32.86.166,147.32.192.34,993
4,2011-08-18 10:19:13.330,0.000,TCP,2.0,169.0,212.24.150.110,147.32.86.166,33426


## Retrait des valeurs na

In [9]:
#nombre de valeurs na dans le dataframe
na_values = len(df_drop) - df_drop.count()
print("A chaque variable correspond le nombre de valeurs na: ")
print(na_values)

A chaque variable correspond le nombre de valeurs na: 
Datetime         0
Durat         1283
Prot          1283
Packets       1283
Bytes         1283
SourceIP      1283
DestIP        1283
DestPort    375948
dtype: int64


In [10]:
#retirer les valeurs NaN de la variable DestPort
df_drop = df_drop[pd.notnull(df_drop['DestPort'])]
df_drop.count()

Datetime    4804903
Durat       4804903
Prot        4804903
Packets     4804903
Bytes       4804903
SourceIP    4804903
DestIP      4804903
DestPort    4804903
dtype: int64

On remarque maintenant que le nombre de lignes est le même pour toutes les variables/colonnes

In [ ]:
#vérification du nouveau nombre de valeurs NaN dans le dataframe
na_values_2 = len(df_drop) - df_drop.count()
print("A chaque variable correspond le nombre de valeurs na: ")
print(na_values_2)

Les variables n'ont plus de valeurs na, cela signifie que les valeurs na étaient communes à toutes ces variables 

### Réduction du jeu de données / retrait des données insignifiantes

In [11]:
#suppression de toutes les lignes où le nombre de paquets est inférieur à 50
df_final = df_drop[df_drop["Packets"]>50]
print(df_final.count())
print("Le jeu de données est-il un dataframe?" , isinstance (df_final, pd.DataFrame))

Datetime    113563
Durat       113563
Prot        113563
Packets     113563
Bytes       113563
SourceIP    113563
DestIP      113563
DestPort    113563
dtype: int64
Le jeu de données est-il un dataframe? True


Le nettoyage de données nous fait passer de 5180851 de lignes à 113563. 

## Transformation du type des variables

In [ ]:
df_final.dtypes

Les variables "Durat" (Durée de connexion), "Packets" (nombre de paquets envoyés) et "Bytes" (nombre de bytes) sont au bon format : "float".
Cependant, il serait préférable de convertir la variable "Datetime" au format adéquat.

In [12]:
df_final = pd.DataFrame(df_final)
df_final["Datetime"] = pd.to_datetime(df_final["Datetime"])
df_final.dtypes

Datetime    datetime64[ns]
Durat              float64
Prot                object
Packets            float64
Bytes              float64
SourceIP            object
DestIP              object
DestPort            object
dtype: object

Etant donné que nous recherchons un comportement inhabituel dans les échanges de paquets, nous procédons au retrait des paquets dont la valeur est inférieure à 50 (valeur "normale")

# Exploration approfondie du jeu de données

### Exploration des données en terme d'échange de paquets et de connexions

In [13]:
#classer le jeu de données par ordre décroissant du nombre de paquets:
df_sort = df_final.sort_values("Packets", ascending=False)
df_sort.head()

,Datetime,Durat,Prot,Packets,Bytes,SourceIP,DestIP,DestPort
3489185,2011-08-18 13:39:04.034,4.999,TCP,110189.0,166813983.0,147.32.127.222,147.32.86.148,36060
3489184,2011-08-18 13:39:04.034,4.999,TCP,102705.0,6998227.0,147.32.86.148,147.32.127.222,80
3495352,2011-08-18 13:39:24.034,4.999,TCP,101740.0,154033984.0,147.32.127.222,147.32.86.148,36060
3490817,2011-08-18 13:39:09.034,4.999,TCP,99887.0,151227470.0,147.32.127.222,147.32.86.148,36060
3495350,2011-08-18 13:39:24.034,4.999,TCP,95485.0,6555058.0,147.32.86.148,147.32.127.222,80


On remarque que sur les 20 plus grandes valeurs de paquets envoyés, un échange s'effectue entre une adresse IP source et une adresse IP de destination qui sont les mêmes et ce plusieurs fois.

In [14]:
#extraire les lignes où le port de destination est 80 (internet)
df_Internet = df_sort.loc[df_sort["DestPort"] == "80"]
print(len(df_Internet.index))
df_Internet.head()

20522


,Datetime,Durat,Prot,Packets,Bytes,SourceIP,DestIP,DestPort
3489184,2011-08-18 13:39:04.034,4.999,TCP,102705.0,6998227.0,147.32.86.148,147.32.127.222,80
3495350,2011-08-18 13:39:24.034,4.999,TCP,95485.0,6555058.0,147.32.86.148,147.32.127.222,80
3490816,2011-08-18 13:39:09.034,4.999,TCP,94286.0,6431668.0,147.32.86.148,147.32.127.222,80
3498708,2011-08-18 13:39:34.034,4.999,TCP,84829.0,5812910.0,147.32.86.148,147.32.127.222,80
3497142,2011-08-18 13:39:29.034,4.999,TCP,82445.0,5654174.0,147.32.86.148,147.32.127.222,80


In [20]:
#extraire les lignes où l'adresse IP de destination est 147.32.96.69:
df_Destination = df_sort.loc[df_sort["DestIP"] == "147.32.96.69"]
print(len(df_Destination.index))
df_Destination.head()

3196


,Datetime,Durat,Prot,Packets,Bytes,SourceIP,DestIP,DestPort
1526235,2011-08-18 11:53:11.663,4.999,UDP,1002.0,788734.0,147.32.84.192,147.32.96.69,0
1526236,2011-08-18 11:53:11.663,4.999,UDP,1002.0,788743.0,147.32.84.204,147.32.96.69,0
1526281,2011-08-18 11:53:11.772,4.998,UDP,1002.0,788724.0,147.32.84.193,147.32.96.69,0
1526344,2011-08-18 11:53:12.066,4.999,UDP,1002.0,788731.0,147.32.84.205,147.32.96.69,0
1547249,2011-08-18 11:54:31.793,4.997,UDP,1000.0,787135.0,147.32.84.165,147.32.96.69,0


L'IP de destination "147.32.127.222" est attaquée 9 fois par l'IP "147.32.86.148" avec un nombre de paquets de l'ordre de plusieurs dizaines de milliers en moins de 34 millièmes de secondes.

In [19]:
#extraire les lignes où l'adresse IP source est 147.32.96.69:
df_Source = df_sort.loc[df_sort["DestIP"] == "147.32.96.69"]
print(len(df_Source.index))
df_Source.head()

3196


,Datetime,Durat,Prot,Packets,Bytes,SourceIP,DestIP,DestPort
1526235,2011-08-18 11:53:11.663,4.999,UDP,1002.0,788734.0,147.32.84.192,147.32.96.69,0
1526236,2011-08-18 11:53:11.663,4.999,UDP,1002.0,788743.0,147.32.84.204,147.32.96.69,0
1526281,2011-08-18 11:53:11.772,4.998,UDP,1002.0,788724.0,147.32.84.193,147.32.96.69,0
1526344,2011-08-18 11:53:12.066,4.999,UDP,1002.0,788731.0,147.32.84.205,147.32.96.69,0
1547249,2011-08-18 11:54:31.793,4.997,UDP,1000.0,787135.0,147.32.84.165,147.32.96.69,0


In [18]:
#compter le nombre de fois où DestIP vaut 147.32.84.59
df_count_147 = df_final[df_final.DestIP=="147.32.84.59"]
print("L'adresse IP 147.32.84.59 effectue :", len(df_count_147), "échanges")
df_count_147.head()

L'adresse IP 147.32.84.59 effectue : 13028 échanges


,Datetime,Durat,Prot,Packets,Bytes,SourceIP,DestIP,DestPort
1,2011-08-18 10:19:13.328,4.995,UDP,617.0,40095.0,82.39.2.249,147.32.84.59,43087
12,2011-08-18 10:19:13.341,4.975,TCP,780.0,1107080.0,192.221.106.126,147.32.84.59,2774
18,2011-08-18 10:19:13.346,4.998,UDP,126.0,8571.0,114.78.14.160,147.32.84.59,43087
38,2011-08-18 10:19:13.385,4.937,UDP,201.0,12870.0,24.4.101.240,147.32.84.59,43087
72,2011-08-18 10:19:13.458,4.743,TCP,496.0,748976.0,74.125.108.208,147.32.84.59,2768


In [21]:
#compter le nombre de fois où DestIP vaut 195.113.232.82
df_count_195 = df_final[df_final.DestIP=="195.113.232.82"]
print("L'adresse IP 195.113.232.828 effectue :", len(df_count_195), "échanges")
df_count_195.head()

L'adresse IP 195.113.232.828 effectue : 35 échanges


,Datetime,Durat,Prot,Packets,Bytes,SourceIP,DestIP,DestPort
36099,2011-08-18 10:21:35.499,1.407,TCP,63.0,12255.0,147.32.86.148,195.113.232.82,80
36120,2011-08-18 10:21:35.543,1.375,TCP,64.0,11724.0,147.32.86.148,195.113.232.82,80
36127,2011-08-18 10:21:35.548,1.371,TCP,56.0,12378.0,147.32.86.148,195.113.232.82,80
36128,2011-08-18 10:21:35.548,1.377,TCP,65.0,12603.0,147.32.86.148,195.113.232.82,80
36130,2011-08-18 10:21:35.549,1.371,TCP,58.0,10805.0,147.32.86.148,195.113.232.82,80


### Subdivision des données sur la variable "Prot"

In [22]:
UDP = df_final[df_final['Prot']== "UDP"]
print("il y a :", len(UDP), "valeurs UDP")
TCP = df_final[df_final['Prot']== "TCP"]
print("il y a :", len(TCP), "valeurs TCP")
ICMP = df_final[df_final['Prot']== "ICMP"]
print("il y a :", len(ICMP), "valeurs ICMP")
PIM = df_final[df_final['Prot']== "PIM"]
print("il y a :", len(PIM), "valeurs PIM")
RTP = df_final[df_final['Prot']== "RTP"]
print("il y a :", len(RTP), "valeurs RTP")
ARP = df_final[df_final['Prot']== "ARP"]
print("il y a :", len(ARP), "valeurs ARP")
RTCP = df_final[df_final['Prot']== "RTCP"]
print("il y a :", len(RTCP), "valeurs RTCP")
IGMP = df_final[df_final['Prot']== "IGMP"]
print("il y a :", len(IGMP), "valeurs IGMP")
IPV6 = df_final[df_final['Prot']== "IPV6"]
print("il y a :", len(IPV6), "valeurs IPV6")
ESP = df_final[df_final['Prot']== "ESP"]
print("il y a :", len(ESP), "valeurs ESP")
LLC = df_final[df_final['Prot']== "LLC"]
print("il y a :", len(LLC), "valeurs LLC")
UDT = df_final[df_final['Prot']== "UDT"]
print("il y a :", len(UDT), "valeurs UDT")

il y a : 14707 valeurs UDP
il y a : 98856 valeurs TCP
il y a : 0 valeurs ICMP
il y a : 0 valeurs PIM
il y a : 0 valeurs RTP
il y a : 0 valeurs ARP
il y a : 0 valeurs RTCP
il y a : 0 valeurs IGMP
il y a : 0 valeurs IPV6
il y a : 0 valeurs ESP
il y a : 0 valeurs LLC
il y a : 0 valeurs UDT


On remarque qu'il n'y a eu des échanges sur le réseau qu'à travers les protocoles:  
- "UDP" : protocole orienté "non connexion".
- "TCP" : protocole orienté "connexion". Le DestIP renvoie un accusé de réception au SourceIP. 


# Export des données réduites au format csv

In [23]:
df_final.to_csv("Netflow_cleandata.csv", sep = ';', index = False)

## Représentation graphique / théorie des graphes

Si l'on souhaite construire un graphe à partir de ces données, il semble évident que les colonnes: 
- "SourceIP" et "DestIP" représenteront les noeuds
- "Packets" représentera le poids des arcs
- Les arcs sont ici représentés par l'existence d'une valeur dans les cases SourceIP et DestIP en même temps.

Il faut donc commencer par récupérer la liste de toutes les adresses IP.

###### Remarque: 
Il y a des adresses IP qui se retrouvent tantôt dans la colonne Source tantôt dans celle de Destination. 
La liste des IP ne doit contenir qu'une fois chaque adresse.

#### Définition du nombre de noeuds dans le graphe

In [24]:
#affichage du nombre d'adresse unique dans chaque colonne
print("le nombre d'adresses IP unique dans DestIP est:",len(df_final.DestIP.unique()))
print("le nombre d'adresses IP unique dans SourceIP est:",len(df_final.SourceIP.unique()))

#Construire la liste des adresses IP uniques
uniq_dest = np.array(df_final.DestIP.unique()) #récupérer les valeurs uniques de la colonne DestIP
uniq_source = np.array(df_final.SourceIP.unique()) #récupérer les valeurs uniques de la colonne SourceIP
uniq_IP = np.concatenate([uniq_dest,uniq_source]) #regrouper toutes les valeurs uniques de DestIP et SourceIP
uniq_IP = np.unique(uniq_IP) #ne garder que les valeurs uniques
print("Il y a au total: ", len(uniq_IP), " adresses IP uniques")


le nombre d'adresses IP unique dans DestIP est: 2349
le nombre d'adresses IP unique dans SourceIP est: 2404
Il y a au total:  2943  adresses IP uniques


Le graphe total compterait donc un total de 2943 noeuds.
Afin de simplifier la visualisation des données, je ne les importerai pas dans leur totalité dans l'IDE de Tulip.
Je visualiserai les 10 graphes issus de la subdivision temporelle.

### Subdivision des données sur la variable "Datetime"

In [25]:
print("l'heure de début est: ", min(df_final["Datetime"]))
print("l'heure de fin est: ", max(df_final["Datetime"]))

l'heure de début est:  2011-08-18 10:19:13.328000
l'heure de fin est:  2011-08-18 15:04:58.815000


On note que les données sont réparties sur 5h le même jour.

In [ ]:
#Séparer la date des heures
df_final_byhour = pd.DataFrame(df_final)
df_final_byhour['Date'] = [d.date() for d in df_final['Datetime']]
df_final_byhour['Heure'] = [d.time() for d in df_final['Datetime']]
del(df_final_byhour['Datetime'])
df_final_byhour.head()

In [27]:
#classer les données par heure croissante:
df_date_sort = df_final.sort_values("Datetime", ascending=True)
df_date_sort.head()

,Datetime,Durat,Prot,Packets,Bytes,SourceIP,DestIP,DestPort
1,2011-08-18 10:19:13.328,4.995,UDP,617.0,40095.0,82.39.2.249,147.32.84.59,43087
2,2011-08-18 10:19:13.329,4.996,UDP,1290.0,1909200.0,147.32.84.59,82.39.2.249,41915
6,2011-08-18 10:19:13.335,4.978,TCP,311.0,70580.0,80.78.79.156,147.32.86.24,31002
7,2011-08-18 10:19:13.335,4.978,UDP,292.0,64319.0,147.32.86.24,151.41.188.39,49621
10,2011-08-18 10:19:13.337,4.988,TCP,204.0,178434.0,188.95.61.42,147.32.86.110,48190


In [29]:
#Séparer le jeu de données en sous jeu de données sur une période de 30 min
data = df_final.set_index("Datetime")



In [36]:
i=0
for n,g in data.groupby(pd.Grouper(freq='30T')):
    name = n.strftime("%Y%m%d%H%M") + ".csv"    
    g.to_csv(name, sep = ";")
    i = i+1

Les données, subdivisées en tranches horaires de 30 minutes et réparties dans des fichiers csv sont dans le dossier.